In [1]:
!pip install conllu torch pandas

In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install gensim numpy

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip show gensim scipy


Name: gensim
Version: 4.3.2
Summary: Python framework for fast Vector Space Modelling
Home-page: https://radimrehurek.com/gensim/
Author: Radim Rehurek
Author-email: me@radimrehurek.com
License: LGPL-2.1-only
Location: c:\users\evika\sample\vikas_env\lib\site-packages
Requires: numpy, scipy, smart-open
Required-by: 
---
Name: scipy
Version: 1.11.0
Summary: Fundamental algorithms for scientific computing in Python
Home-page: https://scipy.org/
Author: 
Author-email: 
License: Copyright (c) 2001-2002 Enthought, Inc. 2003-2023, SciPy Developers.
        All rights reserved.
        
        Redistribution and use in source and binary forms, with or without
        modification, are permitted provided that the following conditions
        are met:
        
        1. Redistributions of source code must retain the above copyright
           notice, this list of conditions and the following disclaimer.
        
        2. Redistributions in binary form must reproduce the above
           cop

In [5]:
pip install --upgrade scipy


     ---------------------------------------- 0.0/60.6 kB ? eta -:--:--
     ---------------------------------------- 60.6/60.6 kB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/46.2 MB ? eta -:--:--
   ---------------------------------------- 0.2/46.2 MB 5.6 MB/s eta 0:00:09
   ---------------------------------------- 0.4/46.2 MB 5.4 MB/s eta 0:00:09
    --------------------------------------- 0.6/46.2 MB 5.1 MB/s eta 0:00:09
    --------------------------------------- 0.9/46.2 MB 5.1 MB/s eta 0:00:09
    --------------------------------------- 1.1/46.2 MB 4.9 MB/s eta 0:00:10
   - -------------------------------------- 1.4/46.2 MB 5.1 MB/s eta 0:00:09
   - -------------------------------------- 1.6/46.2 MB 5.1 MB/s eta 0:00:09
   - -------------------------------------- 1.9/46.2 MB 5.2 MB/s eta 0:00:09
   - -------------------------------------- 2.1/46.2 MB 5.2 MB/s eta 0:00:09
   -- ------------------------------------- 2.4/46.2 MB 5.2 MB/s eta 0:00:09
   -- --

In [6]:
pip install scipy==1.12


     ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
     ---------------------------------------- 60.4/60.4 kB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/46.2 MB ? eta -:--:--
   ---------------------------------------- 0.4/46.2 MB 11.2 MB/s eta 0:00:05
   - -------------------------------------- 1.2/46.2 MB 12.5 MB/s eta 0:00:04
   -- ------------------------------------- 2.8/46.2 MB 19.9 MB/s eta 0:00:03
   ---- ----------------------------------- 4.7/46.2 MB 25.1 MB/s eta 0:00:02
   ------ --------------------------------- 7.0/46.2 MB 30.0 MB/s eta 0:00:02
   -------- ------------------------------- 9.4/46.2 MB 33.6 MB/s eta 0:00:02
   ---------- ----------------------------- 12.4/46.2 MB 50.4 MB/s eta 0:00:01
   ----------- ---------------------------- 13.4/46.2 MB 50.4 MB/s eta 0:00:01
   ----------- ---------------------------- 13.4/46.2 MB 50.4 MB/s eta 0:00:01
   ----------- ---------------------------- 13.4/46.2 MB 50.4 MB/s eta 0:

In [7]:
 
pip install scipy==1.11.0


Reason for being yanked: License Violation



  Using cached scipy-1.11.0-cp310-cp310-win_amd64.whl.metadata (59 kB)
Using cached scipy-1.11.0-cp310-cp310-win_amd64.whl (44.0 MB)
  Attempting uninstall: scipy
    Found existing installation: scipy 1.12.0
    Uninstalling scipy-1.12.0:
      Successfully uninstalled scipy-1.12.0


In [8]:
from conllu import parse_incr
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
import torch.optim as optim
import pandas as pd
import gensim.downloader as api
import numpy as np
from collections import deque

In [9]:
#Data Loading

class DependencyParsingDataset(Dataset):
    def __init__(self, file_path):
        self.sentences = self.load_sentences(file_path)

    def load_sentences(self, file_path):
        all_sentences = []
        with open(file_path, 'r', encoding='utf-8') as file:
            for sentence_data in parse_incr(file):
                transitions = self.get_transitions(sentence_data)
                all_sentences.append(transitions)
        return all_sentences

    def transition_oracle(self, stack, remaining_tokens, tokens):
        if len(stack) < 2:
            return 'SHIFT'
        top_token_idx = stack[-1]
        next_token_idx = remaining_tokens[0] if remaining_tokens else None

        if next_token_idx is not None:
            next_head_idx = tokens[next_token_idx - 1]['head']
            top_head_idx = tokens[top_token_idx - 1]['head']

            if next_head_idx == top_token_idx:
                return 'RIGHT-ARC'
            elif top_head_idx == next_token_idx:
                return 'LEFT-ARC'

        return 'SHIFT'

    def get_transitions(self, tokens):
        transitions = []
        stack = [0]  # Start with ROOT
        remaining_tokens = deque()

        # Handle multi-word tokens and null tokens
        for token in tokens:
            if isinstance(token['id'], tuple) and token['form'] == '-':
                continue  # Ignore null tokens
            elif isinstance(token['id'], tuple):
                remaining_tokens.append(token['id'][0])  # Use first index for multi-word tokens
            else:
                remaining_tokens.append(token['id'])

        arcs = []  # (dependent, head)

        while remaining_tokens:
            action = self.transition_oracle(stack, remaining_tokens, tokens)
            features = self.extract_features(stack, remaining_tokens, tokens)
            transitions.append((features, action))

            if action == 'SHIFT':
                stack.append(remaining_tokens.popleft())
            elif action == 'LEFT-ARC':
                arcs.append((stack[-1], remaining_tokens[0]))
                stack.pop()
            elif action == 'RIGHT-ARC':
                arcs.append((remaining_tokens[0], stack[-1]))
                remaining_tokens.popleft()

        return transitions

    def extract_features(self, stack, remaining_tokens, tokens):
        features = {
            'top_idx': 0, 'next_idx': 0,
            'top_word': 'NULL', 'next_word': 'NULL',
            'top_pos': 'NULL', 'next_pos': 'NULL'
        }

        if stack:
            top_idx = stack[-1]
            top_token = tokens[top_idx - 1]
            features['top_idx'] = top_idx
            features['top_word'] = top_token['form'].lower()
            features['top_pos'] = top_token['upos']

        if remaining_tokens:
            next_idx = remaining_tokens[0]
            next_token = tokens[next_idx - 1]
            features['next_idx'] = next_idx
            features['next_word'] = next_token['form'].lower()
            features['next_pos'] = next_token['upos']

        return features

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence_transitions = self.sentences[idx]
        processed_transitions = []
        for transition in sentence_transitions:
            if len(transition) == 2:
                features, action = transition
                processed_transitions.append((features, action))
            else:
                raise ValueError(f"Expected each transition to have 2 elements, got {len(transition)} elements.")

        return processed_transitions

train_data = DependencyParsingDataset('en_ewt-ud-train.conllu')
dev_data = DependencyParsingDataset('en_ewt-ud-dev.conllu')
test_data = DependencyParsingDataset('en_ewt-ud-test.conllu')

In [10]:
train_data.__getitem__(2)


[({'top_idx': 0,
   'next_idx': 1,
   'top_word': '.',
   'next_word': 'dpa',
   'top_pos': 'PUNCT',
   'next_pos': 'PROPN'},
  'SHIFT'),
 ({'top_idx': 1,
   'next_idx': 2,
   'top_word': 'dpa',
   'next_word': ':',
   'top_pos': 'PROPN',
   'next_pos': 'PUNCT'},
  'SHIFT'),
 ({'top_idx': 2,
   'next_idx': 3,
   'top_word': ':',
   'next_word': 'iraqi',
   'top_pos': 'PUNCT',
   'next_pos': 'ADJ'},
  'SHIFT'),
 ({'top_idx': 3,
   'next_idx': 4,
   'top_word': 'iraqi',
   'next_word': 'authorities',
   'top_pos': 'ADJ',
   'next_pos': 'NOUN'},
  'LEFT-ARC'),
 ({'top_idx': 2,
   'next_idx': 4,
   'top_word': ':',
   'next_word': 'authorities',
   'top_pos': 'PUNCT',
   'next_pos': 'NOUN'},
  'SHIFT'),
 ({'top_idx': 4,
   'next_idx': 5,
   'top_word': 'authorities',
   'next_word': 'announced',
   'top_pos': 'NOUN',
   'next_pos': 'VERB'},
  'LEFT-ARC'),
 ({'top_idx': 2,
   'next_idx': 5,
   'top_word': ':',
   'next_word': 'announced',
   'top_pos': 'PUNCT',
   'next_pos': 'VERB'},
  'LE

In [11]:
#Feature Extraction

def create_pos_vocab(parser_data):
    all_pos_tags = set()
    for sentence_transitions in parser_data:
        for transition in sentence_transitions:
            features, action = transition
            top_pos = features['top_pos']
            next_pos = features['next_pos']
            all_pos_tags.add(top_pos)
            all_pos_tags.add(next_pos)

    pos_to_idx = {pos_tag: idx for idx, pos_tag in enumerate(all_pos_tags)}
    pos_to_idx['<PAD>'] = len(pos_to_idx)

    return pos_to_idx

# Assuming 'DependencyParserData' can be iterated and yields (features, action)
pos_vocab = create_pos_vocab(train_data)

In [12]:
word2VecModel = api.load("word2vec-google-news-300")


In [13]:
#Word Embeddings

def get_word_vector(word, word_embeddings):
    try:
        return word_embeddings[word]
    except KeyError:
        # Attempt to remove apostrophes and retry
        cleaned_word = word.replace("'", "")
        if cleaned_word in word_embeddings:
            return word_embeddings[cleaned_word]
        # Finally, return a zero vector if no suitable word is found
        vector_size = word_embeddings.vector_size
        return np.zeros(vector_size)

In [14]:
#Model Architecture

class DependParser(nn.Module):
    def __init__(self, pos_vocab_size, pos_embed_dim, word_embed_dim, hidden_dim, num_actions):
        super(DependParser, self).__init__()
        self.pos_embedder = nn.Embedding(pos_vocab_size, pos_embed_dim)
        self.lstm = nn.LSTM(input_size=word_embed_dim * 2 + pos_embed_dim * 2, hidden_size=hidden_dim, batch_first=True)
        self.classifier = nn.Linear(hidden_dim, num_actions)

    def forward(self, top_word_embeddings, next_word_embeddings, top_pos_indices, next_pos_indices):
        # Embed POS tags and expand dimensions to match word embeddings
        top_pos_embeddings = self.pos_embedder(top_pos_indices).squeeze(1)
        next_pos_embeddings = self.pos_embedder(next_pos_indices).squeeze(1)

        # Combine all embeddings
        combined_embeddings = torch.cat((top_word_embeddings, next_word_embeddings, top_pos_embeddings, next_pos_embeddings), dim=1)

        # Process with LSTM
        lstm_output, _ = self.lstm(combined_embeddings.unsqueeze(1))

        # Classify the output
        logits = self.classifier(lstm_output[:, -1, :])
        return logits

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

pos_vocab_size = len(pos_vocab)  # from your POS vocabulary
pos_embed_dim = 50  # arbitrary choice, can be tuned
word_embed_dim = 300  # assuming your word embeddings are of size 300
hidden_dim = 128  # hidden dimension of the LSTM
num_actions = 3  # "SHIFT", "RIGHT-ARC", "LEFT-ARC"

# Device configuration

# Initialize model
parser_model = DependParser(pos_vocab_size, pos_embed_dim, word_embed_dim, hidden_dim, num_actions)
parser_model.to(device)

DependParser(
  (pos_embedder): Embedding(19, 50)
  (lstm): LSTM(700, 128, batch_first=True)
  (classifier): Linear(in_features=128, out_features=3, bias=True)
)

In [35]:
#Data Preprocessing
action_to_index = {
    "SHIFT": 0,
    "RIGHT-ARC": 1,
    "LEFT-ARC": 2
}
def create_batches(data_batch, word_embeddings, pos_vocab, action_to_idx):
    # Containers for batch data
    top_word_embeddings, next_word_embeddings = [], []
    top_pos_indices, next_pos_indices = [], []
    action_indices = []

    for sentence_transitions in data_batch:
        for features, action in sentence_transitions:
            # Convert words to embeddings
            top_word_embedding = torch.tensor(get_word_vector(features['top_word'], word_embeddings), dtype=torch.float)
            next_word_embedding = torch.tensor(get_word_vector(features['next_word'], word_embeddings), dtype=torch.float)

            # Convert POS tags to indices
            top_pos_index = pos_vocab[features['top_pos']]
            next_pos_index = pos_vocab[features['next_pos']]

            # Append embeddings and POS indices separately
            top_word_embeddings.append(top_word_embedding)
            next_word_embeddings.append(next_word_embedding)
            top_pos_indices.append(torch.tensor([top_pos_index], dtype=torch.long))
            next_pos_indices.append(torch.tensor([next_pos_index], dtype=torch.long))

            # Convert action to index and append
            action_indices.append(action_to_idx[action])

    # Pad sequences
    top_word_embeddings = pad_sequence(top_word_embeddings, batch_first=True, padding_value=0.0)
    next_word_embeddings = pad_sequence(next_word_embeddings, batch_first=True, padding_value=0.0)
    top_pos_indices = pad_sequence(top_pos_indices, batch_first=True, padding_value=pos_vocab['<PAD>'])
    next_pos_indices = pad_sequence(next_pos_indices, batch_first=True, padding_value=pos_vocab['<PAD>'])
    action_indices = torch.tensor(action_indices, dtype=torch.long)

    return (top_word_embeddings, next_word_embeddings, top_pos_indices, next_pos_indices), action_indices

#train
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, collate_fn=lambda batch: create_batches(batch, word2VecModel, pos_vocab, action_to_index))
val_loader = DataLoader(dev_data, batch_size=32, shuffle=True, collate_fn=lambda batch: create_batches(batch, word2VecModel, pos_vocab, action_to_index))
test_loader = DataLoader(test_data, batch_size=32, shuffle=True, collate_fn=lambda batch: create_batches(batch, word2VecModel, pos_vocab, action_to_index))

for batch in train_loader:
    # Unpack the batch data
    (top_word_embeddings, next_word_embeddings, top_pos_indices, next_pos_indices), action_indices = batch

    # Print shapes and type of data in the batch
    print("Top Word Embeddings Shape:", top_word_embeddings.shape)
    print("Next Word Embeddings Shape:", next_word_embeddings.shape)
    print("Top POS Indices Shape:", top_pos_indices.shape)
    print("Next POS Indices Shape:", next_pos_indices.shape)
    print("Action Indices Shape:", action_indices.shape)
    print("Action Indices:", action_indices)

    # Optionally, break after the first batch to just see one example
    break
#traing setup
num_epochs = 3
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(parser_model.parameters(), lr=0.001)

# Example training loop for one epoch
for epoch in range(num_epochs):
    parser_model.train()  # Set the model to training mode
    epoch_loss = 0

    for (top_word_embeddings, next_word_embeddings, top_pos_indices, next_pos_indices), action_indices in train_loader:
        # Move tensors to the appropriate device
        top_word_embeddings = top_word_embeddings.to(device)
        next_word_embeddings = next_word_embeddings.to(device)
        top_pos_indices = top_pos_indices.to(device)
        next_pos_indices = next_pos_indices.to(device)
        action_indices = action_indices.to(device)

        # Zero gradients
        optimizer.zero_grad()

        # Perform a forward pass through the model
        logits = parser_model(top_word_embeddings, next_word_embeddings, top_pos_indices, next_pos_indices)

        # Calculate loss
        loss = loss_fn(logits, action_indices)

        # Backpropagate the gradients
        loss.backward()

        # Update the parameters
        optimizer.step()

        # Accumulate loss
        epoch_loss += loss.item()

    # Print average loss for the epoch
    avg_loss = epoch_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}")

print("Training complete!")

KeyError: 'AUX'

In [16]:
#model validation
def validate(parser_model, data_loader, loss_fn, device):
    parser_model.eval()  # Set the model to evaluation mode
    epoch_loss = 0
    all_predicted_actions = []
    all_true_actions = []

    with torch.no_grad():
        for (top_word_embeddings, next_word_embeddings, top_pos_indices, next_pos_indices), action_indices in data_loader:
            top_word_embeddings = top_word_embeddings.to(device)
            next_word_embeddings = next_word_embeddings.to(device)
            top_pos_indices = top_pos_indices.to(device)
            next_pos_indices = next_pos_indices.to(device)
            action_indices = action_indices.to(device)

            logits = parser_model(top_word_embeddings, next_word_embeddings, top_pos_indices, next_pos_indices)
            loss = loss_fn(logits, action_indices)
            epoch_loss += loss.item()

            _, predicted_actions = torch.max(logits.data, 1)
            all_predicted_actions.extend(predicted_actions.cpu().numpy())
            all_true_actions.extend(action_indices.cpu().numpy())

    avg_loss = epoch_loss / len(data_loader)
    precision, recall, f1 = compute_metrics(all_predicted_actions, all_true_actions)
    return avg_loss, precision, recall, f1


In [17]:
pip install scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [18]:
from sklearn.metrics import precision_score, recall_score, f1_score
#metrics
def compute_metrics(predicted_actions, true_actions):
    precision = precision_score(true_actions, predicted_actions, average='weighted')
    recall = recall_score(true_actions, predicted_actions, average='weighted')
    f1 = f1_score(true_actions, predicted_actions, average='weighted')
    return precision, recall, f1

In [19]:
# Validate the model on the validation data
val_loss, val_precision, val_recall, val_f1 = validate(parser_model, val_loader, loss_fn, device)
print(f"Validation Loss: {val_loss:.4f}, Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, F1 Score: {val_f1:.4f}")

# Evaluate the model on the test data
test_loss, test_precision, test_recall, test_f1 = validate(parser_model, test_loader, loss_fn, device)
print(f"Test Loss: {test_loss:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1 Score: {test_f1:.4f}")

Validation Loss: 0.5145, Precision: 0.7747, Recall: 0.7794, F1 Score: 0.7761
Test Loss: 0.5132, Precision: 0.7736, Recall: 0.7785, F1 Score: 0.7749


In [37]:
from sklearn.metrics import accuracy_score

def compute_attachment_scores(predicted_arcs, predicted_labels, true_arcs, true_labels):
    correct_arcs = [1 if p == t else 0 for p, t in zip(predicted_arcs, true_arcs)]
    correct_labels = [1 if (p_arc == t_arc and p_label == t_label) else 0
                      for p_arc, p_label, t_arc, t_label in zip(predicted_arcs, predicted_labels, true_arcs, true_labels)]

    UAS = sum(correct_arcs) / len(correct_arcs)
    LAS = sum(correct_labels) / len(correct_labels)

    return UAS, LAS

def validate_with_attachment_scores(model, data_loader, criterion, device):
    model.eval()
    total_loss = 0
    all_predicted_arcs = []
    all_predicted_labels = []
    all_true_arcs = []
    all_true_labels = []

    with torch.no_grad():
        for word_embeddings, pos_tags, actions, true_arcs, true_labels in data_loader:
            word_embeddings, pos_tags, actions = word_embeddings.to(device), pos_tags.to(device), actions.to(device)
            true_arcs, true_labels = true_arcs.to(device), true_labels.to(device)

            outputs = model(word_embeddings, pos_tags)
            loss = criterion(outputs.view(-1, output_dim), actions.view(-1))
            total_loss += loss.item()

            _, predicted_actions = torch.max(outputs, dim=2)

            # Assuming `predicted_actions` contains both arcs and labels in the format (arc, label)
            predicted_arcs = predicted_actions[:, :, 0].cpu().numpy()
            predicted_labels = predicted_actions[:, :, 1].cpu().numpy()

            all_predicted_arcs.extend(predicted_arcs)
            all_predicted_labels.extend(predicted_labels)
            all_true_arcs.extend(true_arcs.cpu().numpy())
            all_true_labels.extend(true_labels.cpu().numpy())


    UAS, LAS = compute_attachment_scores(all_predicted_arcs, all_predicted_labels, all_true_arcs, all_true_labels)
    return  UAS, LAS

# Assuming `val_loader` is defined similarly to `train_loader`
validation_UAS, validation_LAS = validate_with_attachment_scores(parser_model, val_loader, loss_fn, device)
print(f'UAS: {validation_UAS}, LAS: {validation_LAS}')

UAS: 0.813, LAS: 0.803


In [21]:
telugu_train_data = DependencyParsingDataset('te_mtg-ud-train.conllu')
telugu_dev_data = DependencyParsingDataset('te_mtg-ud-dev.conllu')

In [22]:
import gensim
from gensim.models import KeyedVectors

telugu_model_path = "D:\download_new\cc.te.300.vec"  # Use raw string literals or double backslashes
word2vec_telugu_model = KeyedVectors.load_word2vec_format(telugu_model_path, binary=False)

In [23]:
pos_vocab = create_pos_vocab(telugu_train_data)

In [24]:
model_telugu = DependParser(pos_vocab_size, pos_embed_dim, word_embed_dim, hidden_dim, num_actions)
model_telugu.to(device)

DependParser(
  (pos_embedder): Embedding(19, 50)
  (lstm): LSTM(700, 128, batch_first=True)
  (classifier): Linear(in_features=128, out_features=3, bias=True)
)

In [28]:
# Example DataLoader usage
train_loader_telugu = DataLoader(telugu_train_data, batch_size=32, shuffle=True,
                          collate_fn=lambda batch: create_batches(batch, word2vec_telugu_model, pos_vocab, action_to_index))

val_loader_telugu = DataLoader(telugu_dev_data, batch_size=32, shuffle=True,
                          collate_fn=lambda batch: create_batches(batch, word2vec_telugu_model, pos_vocab, action_to_index))
for batch in train_loader_telugu:
    # Unpack the batch data
    (stack_top_embeddings, buffer_first_embeddings, stack_top_pos_indices, buffer_first_pos_indices), actions_indices = batch

    # Print shapes and type of data in the batch
    print("Stack Top Embeddings Shape:", stack_top_embeddings.shape)
    print("Buffer First Embeddings Shape:", buffer_first_embeddings.shape)
    print("Stack Top POS Indices Shape:", stack_top_pos_indices.shape)
    print("Buffer First POS Indices Shape:", buffer_first_pos_indices.shape)
    print("Actions Indices Shape:", actions_indices.shape)
    print("Actions Indices:", actions_indices)

    # Optionally, break after the first batch to just see one example
    break

Stack Top Embeddings Shape: torch.Size([235, 300])
Buffer First Embeddings Shape: torch.Size([235, 300])
Stack Top POS Indices Shape: torch.Size([235, 1])
Buffer First POS Indices Shape: torch.Size([235, 1])
Actions Indices Shape: torch.Size([235])
Actions Indices: tensor([0, 2, 0, 0, 2, 0, 2, 2, 0, 1, 0, 0, 2, 2, 0, 1, 0, 2, 0, 0, 0, 2, 2, 2,
        0, 1, 0, 2, 0, 1, 0, 0, 0, 2, 0, 2, 2, 2, 0, 1, 0, 2, 0, 1, 0, 0, 2, 2,
        0, 1, 0, 2, 0, 0, 0, 2, 2, 2, 0, 1, 0, 2, 0, 1, 0, 0, 2, 0, 2, 2, 0, 1,
        1, 0, 0, 2, 0, 0, 2, 2, 2, 0, 1, 0, 0, 2, 2, 0, 2, 0, 1, 0, 0, 2, 2, 0,
        1, 0, 2, 0, 2, 0, 1, 0, 0, 0, 2, 2, 2, 0, 1, 0, 0, 2, 0, 2, 2, 0, 1, 0,
        0, 2, 0, 1, 2, 2, 0, 1, 0, 0, 0, 2, 2, 2, 0, 1, 0, 0, 2, 2, 0, 1, 0, 0,
        2, 2, 0, 0, 0, 2, 2, 1, 0, 0, 2, 0, 0, 0, 2, 2, 1, 0, 0, 0, 2, 2, 0, 1,
        0, 2, 0, 2, 0, 1, 0, 0, 2, 0, 0, 2, 0, 0, 2, 2, 2, 2, 0, 1, 0, 0, 2, 0,
        2, 0, 2, 2, 0, 1, 0, 2, 0, 1, 0, 2, 0, 1, 0, 2, 0, 1, 0, 2, 0, 0, 2, 2,
        0, 1, 

In [30]:
num_epochs =4
model_telugu.train()  # Set the model to training mode
for epoch in range(num_epochs):
    total_loss = 0
    for (stack_top_embeddings, buffer_first_embeddings, stack_top_pos_indices, buffer_first_pos_indices), actions_indices in train_loader_telugu:
        # Move tensors to the appropriate device
        stack_top_embeddings = stack_top_embeddings.to(device)
        buffer_first_embeddings = buffer_first_embeddings.to(device)
        stack_top_pos_indices = stack_top_pos_indices.to(device)
        buffer_first_pos_indices = buffer_first_pos_indices.to(device)
        actions_indices = actions_indices.to(device)

        # Zero gradients
        optimizer.zero_grad()

        # Perform a forward pass through the model
        outputs = parser_model(stack_top_embeddings, buffer_first_embeddings, stack_top_pos_indices, buffer_first_pos_indices)

        # Calculate loss
        loss = loss_fn(outputs, actions_indices)

        # Backpropagate the gradients
        loss.backward()

        # Update the parameters
        optimizer.step()

        # Accumulate loss
        total_loss += loss.item()

    # Print average loss for the epoch
    avg_loss = total_loss / len(train_loader_telugu)
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}")

print("Training complete!")

Epoch 1/4, Loss: 0.6654
Epoch 2/4, Loss: 0.3097
Epoch 3/4, Loss: 0.2834
Epoch 4/4, Loss: 0.2632
Training complete!


In [36]:
# Validate the model on the validation data
val_loss, val_precision, val_recall, val_f1 = validate(model_telugu, val_loader_telugu, loss_fn, device)
print(f"Validation Loss: {val_loss:.4f}, Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, F1 Score: {val_f1:.4f}")

# Evaluate the model on the test data
test_loss, test_precision, test_recall, test_f1 = validate(model_telugu, train_loader_telugu, loss_fn, device)
print(f"Test Loss: {test_loss:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1 Score: {test_f1:.4f}")

Validation Loss: 1.1142, Precision: 0.7375, Recall: 0.3703, F1 Score: 0.2155
Test Loss: 1.1115, Precision: 0.6742, Recall: 0.3692, F1 Score: 0.2193


In [39]:
UAS, vLAS = validate_with_attachment_scores(model_telugu, val_loader_telugu, loss_fn, device)
print(f'UAS: {validation_UAS}, LAS: {validation_LAS}')

UAS: 0.87, LAS: 0.843
